In [1]:
# !pip install opencv-python scikit-learn pandas numpy matplotlib tensorflow-addons
# !pip install tensorflow==2.12 keras==2.12
# !pip install transformers datasets torchvision

In [2]:
import os
import kagglehub
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Conv2D, Multiply, GlobalAveragePooling2D, Dense, Input, Softmax, Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import kagglehub

2025-06-26 17:36:13.016058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750959373.549138      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750959373.683010      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from transformers import ViTImageProcessor, ViTForImageClassification
import torch
from datasets import load_dataset
from PIL import Image
import requests
from sklearn.metrics import classification_report
import torch
from torch.optim import AdamW 
from transformers import ViTFeatureExtractor
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [4]:
path = kagglehub.dataset_download("adityamahimkar/iqothnccd-lung-cancer-dataset")
print("Path to dataset files:", path)
path = kagglehub.dataset_download("justinkirby/the-cancer-imaging-archive-lidcidri")
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/iqothnccd-lung-cancer-dataset
Path to dataset files: /kaggle/input/the-cancer-imaging-archive-lidcidri


In [5]:
data_dir = '/kaggle/input/iqothnccd-lung-cancer-dataset/The IQ-OTHNCCD lung cancer dataset/The IQ-OTHNCCD lung cancer dataset'
test_dir = '/kaggle/input/iqothnccd-lung-cancer-dataset/Test cases'

In [ ]:
img_size = (224, 224)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = val_datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 878 images belonging to 3 classes.
Found 219 images belonging to 3 classes.


In [7]:
print(train_generator.class_indices)

{'Bengin cases': 0, 'Malignant cases': 1, 'Normal cases': 2}


In [8]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

y_train = train_generator.classes  

#Compute class weightss
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

# Convert to dict for Keras
class_weights_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weights_dict)

Class Weights: {0: 3.048611111111111, 1: 0.651818856718634, 2: 0.8788788788788788}


In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from transformers import ViTImageProcessor
import cv2

processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')

def custom_preprocess(img):
    # Convert PIL Image to numpy array
    img_np = np.array(img)
    if img_np.ndim == 3 and img_np.shape[2] == 3:
        img_np = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)
    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_np = clahe.apply(img_np)
    # Bicubic interpolation (resize)
    img_np = cv2.resize(img_np, (224, 224), interpolation=cv2.INTER_CUBIC)
    # Gaussian Blurring
    img_np = cv2.GaussianBlur(img_np, (5,5), 0)
    # Thresholding
    _, img_np = cv2.threshold(img_np, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Erosion
    kernel = np.ones((3,3), np.uint8)
    img_np = cv2.erode(img_np, kernel, iterations=1)
    # Dilation
    img_np = cv2.dilate(img_np, kernel, iterations=1)
    # Convert back to PIL Image
    return Image.fromarray(img_np)

# Update your train_transform and val_transform
train_transform = transforms.Compose([
    transforms.Lambda(custom_preprocess),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Convert to 3 channels if needed
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

val_transform = transforms.Compose([
    transforms.Lambda(custom_preprocess),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Convert to 3 channels if needed
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std)
])

full_dataset = datasets.ImageFolder(root=data_dir)

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_transform

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [10]:
num_classes = len(full_dataset.classes)

model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=num_classes
)

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import copy

best_val_loss = float('inf')
best_model_state = None

In [12]:
import torch
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


train_labels = [full_dataset.samples[i][1] for i in train_dataset.indices]


class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

optimizer = AdamW(model.parameters(), lr=5e-5)

best_val_loss = float('inf')
best_model_state = None

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import torch
from torch.utils.data import Subset
import copy

# Prepare labels for stratification
all_labels = [sample[1] for sample in full_dataset.samples]
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 1
best_fold_accuracies = []

for train_idx, val_idx in skf.split(np.zeros(len(all_labels)), all_labels):
    print(f"\n===== Fold {fold} =====")
    train_subset = Subset(full_dataset, train_idx)
    val_subset = Subset(full_dataset, val_idx)
    # Set transforms
    train_subset.dataset.transform = train_transform
    val_subset.dataset.transform = val_transform

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

    # Compute class weights for this fold
    train_labels_fold = [all_labels[i] for i in train_idx]
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=np.unique(train_labels_fold),
        y=train_labels_fold
    )
    class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
    loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)

    # Re-initialize model for each fold
    model = ViTForImageClassification.from_pretrained(
        'google/vit-base-patch16-224-in21k',
        num_labels=num_classes
    ).to(device)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    best_val_loss = float('inf')
    best_model_state = None

    for epoch in range(10):  # You can adjust epochs
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs).logits
            loss = loss_fn(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        model.eval()
        val_loss = 0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs).logits
                loss = loss_fn(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        avg_val_loss = val_loss / len(val_loader)
        accuracy = correct / total
        print(f"Epoch {epoch+1}, Train Loss: {total_loss:.4f}")
        print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = copy.deepcopy(model.state_dict())
            best_fold_accuracy = accuracy
            print("✅ Best model updated.")
    # Save best model for this fold
    torch.save(best_model_state, f"best_vit_model_fold{fold}.pth")
    print(f"Best model for fold {fold} saved to 'best_vit_model_fold{fold}.pth'")
    best_fold_accuracies.append(best_fold_accuracy)
    fold += 1

print("Cross-validation complete.")
print("Fold Accuracies:", best_fold_accuracies)
print("Mean Accuracy:", np.mean(best_fold_accuracies))

Epoch 1, Train Loss: 20.0034
Validation Loss: 0.5446, Accuracy: 0.9000
✅ Best model updated.
Epoch 2, Train Loss: 9.5773
Validation Loss: 0.4553, Accuracy: 0.9364
✅ Best model updated.
Epoch 3, Train Loss: 5.6990
Validation Loss: 0.2080, Accuracy: 0.9636
✅ Best model updated.
Epoch 4, Train Loss: 3.3641
Validation Loss: 0.1794, Accuracy: 0.9545
✅ Best model updated.
Epoch 5, Train Loss: 2.5548
Validation Loss: 0.2909, Accuracy: 0.8773
Epoch 6, Train Loss: 1.7609
Validation Loss: 0.0886, Accuracy: 0.9909
✅ Best model updated.
Epoch 7, Train Loss: 1.2374
Validation Loss: 0.0847, Accuracy: 0.9773
✅ Best model updated.
Epoch 8, Train Loss: 0.9194
Validation Loss: 0.0834, Accuracy: 0.9818
✅ Best model updated.
Epoch 9, Train Loss: 1.0352
Validation Loss: 0.1589, Accuracy: 0.9727
Epoch 10, Train Loss: 0.6506
Validation Loss: 0.0764, Accuracy: 0.9818
✅ Best model updated.
Epoch 11, Train Loss: 0.5371
Validation Loss: 0.0719, Accuracy: 0.9909
✅ Best model updated.
Epoch 12, Train Loss: 0.4772


In [15]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        logits = model(inputs).logits
        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))
accuracy = accuracy_score(all_labels, all_preds)
print(f"Accuracy: {accuracy:.4f}")

                 precision    recall  f1-score   support

   Bengin cases       0.97      0.97      0.97        31
Malignant cases       1.00      1.00      1.00       104
   Normal cases       0.99      0.99      0.99        85

       accuracy                           0.99       220
      macro avg       0.99      0.99      0.99       220
   weighted avg       0.99      0.99      0.99       220

Accuracy: 0.9909
